# Project: Part of Speech Tagging with Hidden Markov Models 
---
### Introduction

Part of speech tagging is the process of determining the syntactic category of a word from the words in its surrounding context. It is often used to help disambiguate natural language phrases because it can be done quickly with high accuracy. Tagging can be used for many NLP tasks like determining correct pronunciation during speech synthesis (for example, _dis_-count as a noun vs dis-_count_ as a verb), for information retrieval, and for word sense disambiguation.

In this notebook, you'll use the [Pomegranate](http://pomegranate.readthedocs.io/) library to build a hidden Markov model for part of speech tagging using a "universal" tagset. Hidden Markov models have been able to achieve [>96% tag accuracy with larger tagsets on realistic text corpora](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf). Hidden Markov models have also been used for speech recognition and speech generation, machine translation, gene recognition for bioinformatics, and human gesture recognition for computer vision, and more. 

![](_post-hmm.png)

The notebook already contains some code to get you started. You only need to add some new functionality in the areas indicated to complete the project; you will not need to modify the included code beyond what is requested. Sections that begin with **'IMPLEMENTATION'** in the header indicate that you must provide code in the block that follows. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully!

<div class="alert alert-block alert-info">
**Note:** Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You must then **export the notebook** by running the last cell in the notebook, or by using the menu above and navigating to **File -> Download as -> HTML (.html)** Your submissions should include both the `html` and `ipynb` files.
</div>

<div class="alert alert-block alert-info">
**Note:** Code and Markdown cells can be executed using the `Shift + Enter` keyboard shortcut. Markdown cells can be edited by double-clicking the cell to enter edit mode.
</div>

### The Road Ahead
You must complete Steps 1-3 below to pass the project. The section on Step 4 includes references & resources you can use to further explore HMM taggers.

- [Step 1](#Step-1:-Read-and-preprocess-the-dataset): Review the provided interface to load and access the text corpus
- [Step 2](#Step-2:-Build-a-Most-Frequent-Class-tagger): Build a Most Frequent Class tagger to use as a baseline
- [Step 3](#Step-3:-Build-an-HMM-tagger): Build an HMM Part of Speech tagger and compare to the MFC baseline
- [Step 4](#Step-4:-[Optional]-Improving-model-performance): (Optional) Improve the HMM tagger

<div class="alert alert-block alert-warning">
**Note:** Make sure you have selected a **Python 3** kernel in Workspaces or the hmm-tagger conda environment if you are running the Jupyter server on your own machine.
</div>

In [1]:
# Jupyter "magic methods" -- only need to be run once per kernel restart
%load_ext autoreload
%aimport helpers, tests
%autoreload 1

In [2]:
# import python modules -- this cell needs to be run again if you make changes to any of the files
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

## Step 1: Read and preprocess the dataset
---
We'll start by reading in a text corpus and splitting it into a training and testing dataset. The data set is a copy of the [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) (originally from the [NLTK](https://www.nltk.org/) library) that has already been pre-processed to only include the [universal tagset](https://arxiv.org/pdf/1104.2086.pdf). You should expect to get slightly higher accuracy using this simplified tagset than the same model would achieve on a larger tagset like the full [Penn treebank tagset](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html), but the process you'll follow would be the same.

The `Dataset` class provided in helpers.py will read and parse the corpus. You can generate your own datasets compatible with the reader by writing them to the following format. The dataset is stored in plaintext as a collection of words and corresponding tags. Each sentence starts with a unique identifier on the first line, followed by one tab-separated word/tag pair on each following line. Sentences are separated by a single blank line.

Example from the Brown corpus. 
```
b100-38532
Perhaps	ADV
it	PRON
was	VERB
right	ADJ
;	.
;	.

b100-35577
...
```

In [3]:
data = Dataset("tags-universal.txt", "brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 57340 sentences in the corpus.
There are 45872 sentences in the training set.
There are 11468 sentences in the testing set.


In [4]:
data.X[0]

('Mr.',
 'Podger',
 'had',
 'thanked',
 'him',
 'gravely',
 ',',
 'and',
 'now',
 'he',
 'made',
 'use',
 'of',
 'the',
 'advice',
 '.')

### The Dataset Interface

You can access (mostly) immutable references to the dataset through a simple interface provided through the `Dataset` class, which represents an iterable collection of sentences along with easy access to partitions of the data for training & testing. Review the reference below, then run and review the next few cells to make sure you understand the interface before moving on to the next step.

```
Dataset-only Attributes:
    training_set - reference to a Subset object containing the samples for training
    testing_set - reference to a Subset object containing the samples for testing

Dataset & Subset Attributes:
    sentences - a dictionary with an entry {sentence_key: Sentence()} for each sentence in the corpus
    keys - an immutable ordered (not sorted) collection of the sentence_keys for the corpus
    vocab - an immutable collection of the unique words in the corpus
    tagset - an immutable collection of the unique tags in the corpus
    X - returns an array of words grouped by sentences ((w11, w12, w13, ...), (w21, w22, w23, ...), ...)
    Y - returns an array of tags grouped by sentences ((t11, t12, t13, ...), (t21, t22, t23, ...), ...)
    N - returns the number of distinct samples (individual words or tags) in the dataset

Methods:
    stream() - returns an flat iterable over all (word, tag) pairs across all sentences in the corpus
    __iter__() - returns an iterable over the data as (sentence_key, Sentence()) pairs
    __len__() - returns the nubmer of sentences in the dataset
```

For example, consider a Subset, `subset`, of the sentences `{"s0": Sentence(("See", "Spot", "run"), ("VERB", "NOUN", "VERB")), "s1": Sentence(("Spot", "ran"), ("NOUN", "VERB"))}`. The subset will have these attributes:

```
subset.keys == {"s1", "s0"}  # unordered
subset.vocab == {"See", "run", "ran", "Spot"}  # unordered
subset.tagset == {"VERB", "NOUN"}  # unordered
subset.X == (("Spot", "ran"), ("See", "Spot", "run"))  # order matches .keys
subset.Y == (("NOUN", "VERB"), ("VERB", "NOUN", "VERB"))  # order matches .keys
subset.N == 7  # there are a total of seven observations over all sentences
len(subset) == 2  # because there are two sentences
```

<div class="alert alert-block alert-info">
**Note:** The `Dataset` class is _convenient_, but it is **not** efficient. It is not suitable for huge datasets because it stores multiple redundant copies of the same data.
</div>

#### Sentences

`Dataset.sentences` is a dictionary of all sentences in the training corpus, each keyed to a unique sentence identifier. Each `Sentence` is itself an object with two attributes: a tuple of the words in the sentence named `words` and a tuple of the tag corresponding to each word named `tags`.

In [7]:
key = 'b100-38532'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-38532
words:
	('Perhaps', 'it', 'was', 'right', ';', ';')
tags:
	('ADV', 'PRON', 'VERB', 'ADJ', '.', '.')


<div class="alert alert-block alert-info">
**Note:** The underlying iterable sequence is **unordered** over the sentences in the corpus; it is not guaranteed to return the sentences in a consistent order between calls. Use `Dataset.stream()`, `Dataset.keys`, `Dataset.X`, or `Dataset.Y` attributes if you need ordered access to the data.
</div>

#### Counting Unique Elements

You can access the list of unique words (the dataset vocabulary) via `Dataset.vocab` and the unique list of tags via `Dataset.tagset`.

In [8]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))

assert data.N == data.training_set.N + data.testing_set.N, \
       "The number of training + test samples should sum to the total number of samples"

There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 928458 samples of 50536 unique words in the training set.
There are 232734 samples of 25112 unique words in the testing set.
There are 5521 words in the test set that are missing in the training set.


#### Accessing word and tag Sequences
The `Dataset.X` and `Dataset.Y` attributes provide access to ordered collections of matching word and tag sequences for each sentence in the dataset.

In [9]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')



#### Accessing (word, tag) Samples
The `Dataset.stream()` method returns an iterator that chains together every pair of (word, tag) entries across all sentences in the entire corpus.

In [10]:
# use Dataset.stream() (word, tag) samples for the entire corpus
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 5: break


Stream (word, tag) pairs:

	 ('Mr.', 'NOUN')
	 ('Podger', 'NOUN')
	 ('had', 'VERB')
	 ('thanked', 'VERB')
	 ('him', 'PRON')
	 ('gravely', 'ADV')
	 (',', '.')



For both our baseline tagger and the HMM model we'll build, we need to estimate the frequency of tags & words from the frequency counts of observations in the training corpus. In the next several cells you will complete functions to compute the counts of several sets of counts. 

## Step 2: Build a Most Frequent Class tagger
---

Perhaps the simplest tagger (and a good baseline for tagger performance) is to simply choose the tag most frequently assigned to each word. This "most frequent class" tagger inspects each observed word in the sequence and assigns it the label that was most often assigned to that word in the corpus.

### IMPLEMENTATION: Pair Counts

Complete the function below that computes the joint frequency counts for two input sequences.

In [11]:
len(data.vocab)

56057

In [12]:
def pair_counts(sequences_A, sequences_B):
    """Return a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    you should return a dictionary such that pair_counts[NOUN][time] == 1244
    """
    # TODO: Finish this function!
    big_dict = dict([(key, {}) for key in data.tagset])
    for set_x, set_y in zip(sequences_A, sequences_B):
        for x, y in zip(set_x, set_y):
            if x in big_dict[y]:
                big_dict[y][x] +=1
            else:
                big_dict[y].update({x:1})
    return big_dict
# Calculate C(t_i, w_i)
emission_counts = pair_counts(data.X, data.Y)# TODO: YOUR CODE HERE)

assert len(emission_counts) == 12, \
       "Uh oh. There should be 12 tags in your dictionary."
assert max(emission_counts["NOUN"], key=emission_counts["NOUN"].get) == 'time', \
       "Hmmm...'time' is expected to be the most common NOUN."
HTML('<div class="alert alert-block alert-success">Your emission counts look good!</div>')

### IMPLEMENTATION: Most Frequent Class Tagger

Use the `pair_counts()` function and the training dataset to find the most frequent class label for each word in the training data, and populate the `mfc_table` below. The table keys should be words, and the values should be the appropriate tag string.

The `MFCTagger` class is provided to mock the interface of Pomegranite HMM models so that they can be used interchangeably.

In [87]:
def create_table():
    ''' '''
    new_dict ={}
    for key in data.training_set.vocab:
        new_dict.update({key : {}})
    for x ,y  in data.training_set.stream():
        if y in new_dict[x]:
            new_dict[x][y] +=1
        else:
            new_dict[x].update({y:1})
    final_dict= {}
    for key in new_dict:
        final_dict.update({key:max(new_dict[key], key= lambda k: new_dict[key][k])})
    return final_dict

In [90]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label most frequently assigned to that word
from collections import namedtuple

FakeState = namedtuple("FakeState", "name")

class MFCTagger:
    # NOTE: You should not need to modify this class or any of its methods
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))


# TODO: calculate the frequency of each tag being assigned to each word (hint: similar, but not
# the same as the emission probabilities) and use it to fill the mfc_table

word_counts = pair_counts(data.X, data.Y) # TODO: YOUR CODE HERE)

mfc_table = create_table() # TODO: YOUR CODE HERE

# DO NOT MODIFY BELOW THIS LINE
mfc_model = MFCTagger(mfc_table) # Create a Most Frequent Class tagger instance

assert len(mfc_table) == len(data.training_set.vocab), ""
assert all(k in data.training_set.vocab for k in mfc_table.keys()), ""
assert sum(int(k not in mfc_table) for k in data.testing_set.vocab) == 5521, ""
HTML('<div class="alert alert-block alert-success">Your MFC tagger has all the correct words!</div>')

### Making Predictions with a Model
The helper functions provided below interface with Pomegranate network models & the mocked MFCTagger to take advantage of the [missing value](http://pomegranate.readthedocs.io/en/latest/nan.html) functionality in Pomegranate through a simple sequence decoding function. Run these functions, then run the next cell to see some of the predictions made by the MFC tagger.

In [91]:
def replace_unknown(sequence):
    """Return a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. Pomegranate will ignore these values
    during computation.
    """
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  # do not show the start/end state predictions

### Example Decoding Sequences with MFC Tagger

In [92]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', '<MISSING>', 'ADP', 'ADJ', 'ADJ', 

### Evaluating Model Accuracy

The function below will evaluate the accuracy of the MFC tagger on the collection of all sentences from a text corpus. 

In [93]:
def accuracy(X, Y, model):
    """Calculate the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

#### Evaluate the accuracy of the MFC tagger
Run the next cell to evaluate the accuracy of the tagger on the training and test corpus.

In [94]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

assert mfc_training_acc >= 0.955, "Uh oh. Your MFC accuracy on the training set doesn't look right."
assert mfc_testing_acc >= 0.925, "Uh oh. Your MFC accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your MFC tagger accuracy looks correct!</div>')

training accuracy mfc_model: 95.72%
testing accuracy mfc_model: 93.01%


## Step 3: Build an HMM tagger
---
The HMM tagger has one hidden state for each possible tag, and parameterized by two distributions: the emission probabilties giving the conditional probability of observing a given **word** from each hidden state, and the transition probabilities giving the conditional probability of moving between **tags** during the sequence.

We will also estimate the starting probability distribution (the probability of each **tag** being the first tag in a sequence), and the terminal probability distribution (the probability of each **tag** being the last tag in a sequence).

The maximum likelihood estimate of these distributions can be calculated from the frequency counts as described in the following sections where you'll implement functions to count the frequencies, and finally build the model. The HMM model will make predictions according to the formula:

$$t_i^n = \underset{t_i^n}{\mathrm{argmin}} \prod_{i=1}^n P(w_i|t_i) P(t_i|t_{i-1})$$

Refer to Speech & Language Processing [Chapter 10](https://web.stanford.edu/~jurafsky/slp3/10.pdf) for more information.

### IMPLEMENTATION: Unigram Counts

Complete the function below to estimate the co-occurrence frequency of each symbol over all of the input sequences. The unigram probabilities in our HMM model are estimated from the formula below, where N is the total number of samples in the input. (You only need to compute the counts for now.)

$$P(tag_1) = \frac{C(tag_1)}{N}$$

In [105]:
def unigram_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequence list that
    counts the number of occurrences of the value in the sequences list. The sequences
    collection should be a 2-dimensional array.
    
    For example, if the tag NOUN appears 275558 times over all the input sequences,
    then you should return a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    # TODO: Finish this function!
    unigram_dict = {}
    for key in data.training_set.tagset:
        unigram_dict.update({key : 0})
    for tags in sequences:
        for tag in tags:
            unigram_dict[tag] +=1
    return unigram_dict

# TODO: call unigram_counts with a list of tag sequences from the training set
tag_unigrams = unigram_counts(data.Y)

assert set(tag_unigrams.keys()) == data.training_set.tagset, \
       "Uh oh. It looks like your tag counts doesn't include all the tags!"
assert min(tag_unigrams, key=tag_unigrams.get) == 'X', \
       "Hmmm...'X' is expected to be the least common class"
assert max(tag_unigrams, key=tag_unigrams.get) == 'NOUN', \
       "Hmmm...'NOUN' is expected to be the most common class"
HTML('<div class="alert alert-block alert-success">Your tag unigrams look good!</div>')

### IMPLEMENTATION: Bigram Counts

Complete the function below to estimate the co-occurrence frequency of each pair of symbols in each of the input sequences. These counts are used in the HMM model to estimate the bigram probability of two tags from the frequency counts according to the formula: $$P(tag_2|tag_1) = \frac{C(tag_2|tag_1)}{C(tag_2)}$$


In [192]:
def bigram_counts(sequences):
    """Return a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then you should
    return a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """
    
    # TODO: Finish this function!
    biagram_dict = {}
    for tag1 in data.tagset:
        for tag2 in data.tagset:
            biagram_dict.update({(tag1, tag2):0})
    for text in data.Y:
        for i in range(len(text) - 1):
            biagram_dict[tuple((text[i], text[i+1]))] +=1        
    return biagram_dict
    

# TODO: call bigram_counts with a list of tag sequences from the training set
tag_bigrams = bigram_counts(data.training_set.sentences)

assert len(tag_bigrams) == 144, \
       "Uh oh. There should be 144 pairs of bigrams (12 tags x 12 tags)"
assert min(tag_bigrams, key=tag_bigrams.get) in [('X', 'NUM'), ('PRON', 'X')], \
       "Hmmm...The least common bigram should be one of ('X', 'NUM') or ('PRON', 'X')."
assert max(tag_bigrams, key=tag_bigrams.get) in [('DET', 'NOUN')], \
       "Hmmm...('DET', 'NOUN') is expected to be the most common bigram."
HTML('<div class="alert alert-block alert-success">Your tag bigrams look good!</div>')

### IMPLEMENTATION: Sequence Starting Counts
Complete the code below to estimate the bigram probabilities of a sequence starting with each tag.

In [201]:
def starting_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then you should return a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    # TODO: Finish this function!
    start_dict = {}
    for key in data.tagset:
        start_dict.update({key : 0})
    for tag in sequences:
        start_dict[tag[0]] +=1
    return start_dict
# TODO: Calculate the count of each tag starting a sequence
tag_starts = starting_counts(data.Y)

assert len(tag_starts) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_starts, key=tag_starts.get) == 'X', "Hmmm...'X' is expected to be the least common starting bigram."
assert max(tag_starts, key=tag_starts.get) == 'DET', "Hmmm...'DET' is expected to be the most common starting bigram."
HTML('<div class="alert alert-block alert-success">Your starting tag counts look good!</div>')

### IMPLEMENTATION: Sequence Ending Counts
Complete the function below to estimate the bigram probabilities of a sequence ending with each tag.

In [204]:
def ending_counts(sequences):
    """Return a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then you should return a
    dictionary such that your_starting_counts[DET] == 18
    """
    # TODO: Finish this function!
    end_dict = {}
    for key in data.tagset:
        end_dict.update({key : 0})
    for tag in sequences:
        end_dict[tag[len(tag)-1]] +=1
    return end_dict

# TODO: Calculate the count of each tag ending a sequence
tag_ends = ending_counts(data.Y)

assert len(tag_ends) == 12, "Uh oh. There should be 12 tags in your dictionary."
assert min(tag_ends, key=tag_ends.get) in ['X', 'CONJ'], "Hmmm...'X' or 'CONJ' should be the least common ending bigram."
assert max(tag_ends, key=tag_ends.get) == '.', "Hmmm...'.' is expected to be the most common ending bigram."
HTML('<div class="alert alert-block alert-success">Your ending tag counts look good!</div>')

### IMPLEMENTATION: Basic HMM Tagger
Use the tag unigrams and bigrams calculated above to construct a hidden Markov tagger.

- Add one state per tag
    - The emission distribution at each state should be estimated with the formula: $P(w|t) = \frac{C(t, w)}{C(t)}$
- Add an edge from the starting state `basic_model.start` to each tag
    - The transition probability should be estimated with the formula: $P(t|start) = \frac{C(start, t)}{C(start)}$
- Add an edge from each tag to the end state `basic_model.end`
    - The transition probability should be estimated with the formula: $P(end|t) = \frac{C(t, end)}{C(t)}$
- Add an edge between _every_ pair of tags
    - The transition probability should be estimated with the formula: $P(t_2|t_1) = \frac{C(t_1, t_2)}{C(t_1)}$

In [206]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

In [ ]:
s1 = State()

In [229]:
for key in emission_counts:
    for word in emission_counts[key]:
        print(word, emission_counts[key][word])

per 369
durin' 3
As 492
above 169
Till 3
During 132
Whereas 10
upon 431
Along 8
Until 28
except 163
concerning 59
towards 62
Respecting 1
Besides 16
considerin' 1
After 246
inside 77
Than 2
lest 17
unlike 33
till 45
tho' 1
save 5
whether 258
with-but-after 1
Vs. 1
nearer 2
so's 4
: 20
behind 217
But 3
times 3
despite 66
under 632
Fur 1
respecting 1
Next 5
t'hi-im 1
including 165
On 345
According 29
but 128
Despite 38
Upon 20
Depending 1
Unlike 9
o' 15
seeing 1
Save 2
'ceptin' 1
pending 4
lahk 1
around 321
underneath 6
while 480
astride 3
to 10985
albeit 2
Below 7
At 409
notwithstanding 2
- 72
round 4
'bout 1
in 19066
Pending 1
Except 10
up 175
as 5656
over 820
involving 3
providing 3
unto 16
Including 1
inter 1
between 715
out 436
beside 76
Whether 28
With 277
Once 28
whereas 31
Toward 6
Over 21
by 5044
across 250
befoh 1
depending 28
on 5837
Supposing 1
Against 9
than 1788
Out 18
plus 16
By 200
'round 1
vis-a-vis 2
Uh 1
next 37
until 433
pursuant 20
Without 42
without 541
then 2
follo

1887 4
31730 1
1966 4
five-and-a-half 1
203 7
1300 1
1745 1
36-A 1
81 3
1801 4
7,484,268 1
1891 4
140,000 1
230 4
1797 3
5.6 2
sixteen 18
10:30 2
1307 1
268,900 1
Forty-six 2
ten-twelve 1
2:34.2 1
98 3
-.10 1
1200 1
15.0 1
21-2 1
1955 28
0.154 1
1644 1
1690 1
1-3 1
196 2
7-1 6
2,418 1
1310 1
182 1
3300 1
forty-eight 1
1806 1
1820 3
2.44 1
790 1
1639 1
4,122,354 1
60 46
4.0 1
96 5
17.3 1
1720 1
1777 1
Twenty 7
three 553
1925 9
fifty-five 2
1565 2
2.55 1
2 445
1911-1912 1
1642 1
0.4 4
1909-10 1
7.2 1
1639-40 1
920 1
108 4
1981 1
2500 1
2:31.3-:35.3 1
1688 1
1959-60 2
6- 1
two-to-three 1
1692 1
3.190 1
1,338,000 1
'51 1
2433 1
113 3
1640 1
Eighty-Four 3
1947 14
5612 1
hundred-odd 2
'38 1
4.7 1
1687 1
147,000 1
billion 62
127 1
Thirteen 2
11,330 1
13 49
1816 1
87 2
3,500 3
2:33.3 1
55,000 1
430,000 1
48,000 1
1,571 1
2:36 12
151 1
188 2
1514 1
1783 3
1958-60 1
1,450,000 1
389 1
275-300 1
120,000 1
367 1
'60 1
6(j) 1
2:05.2 1
2:33 3
5- 1
08 2
25-30 1
710 1
643 1
1986 3
9.3 1
0.2 4
260 5
9N 

a.m. 27
large 5
improbably 1
blindly 8
nakedly 1
upward 24
weekly 7
scholastically 1
furthermore 3
fantastically 2
warmly 7
slimly 1
keenly 3
Less 5
electronically 1
Happily 1
sweet 1
yet 226
'nuff 1
thrice 1
vaguely 17
Wisely 1
uselessly 3
noticeably 2
majestically 1
customarily 4
quicker 1
counter-clockwise 1
diagonally 4
Also 70
clinically 1
A.M. 13
thenceforth 1
privately 10
naively 1
Entirely 1
honorably 3
subconsciously 4
Functionally 1
some 19
analytically 1
Very 24
louder 1
thereto 11
insofar 5
altruistically 1
gust 1
later 304
atonally 1
horrifyingly 1
meticulously 6
casually 13
whereof 8
believably 1
soothingly 1
according 10
Moreover 67
fiercely 4
yonder 1
mightily 1
capably 1
tenaciously 1
correspondingly 2
normally 34
admiringly 1
o'clock 39
Traditionally 1
glibly 4
therefrom 5
Obligingly 1
natch 1
huskily 1
uptown 1
unreasonably 1
temptingly 1
Acourse 1
indecisively 1
awake 7
extensively 10
industriously 2
partially 25
theoretically 4
Probably 25
sinuously 1
analogously 1

Great's 1
in 465
Blackwell's 1
Morning 1
Baseball's 1
up 1699
She'll 6
Rockabye 1
thet's 1
buzz-buzz-buzz 1
Please 7
what're 1
Myra's 1
Uh-huh 4
Arlene's 1
many 20
Kaboom 1
Yalagaloo 1
Aye-yah-ah-ah 1
damn 3
they'd 24
There'll 1
Wow 1
she'll 4
Sonuvabitch 1
Yesiree 1
Uh 3
we're 35
a-gracious 1
Jee-sus 1
Nobody's 2
Goddammit 1
fire's 2
you've 43
wife's 1
You'll 30
Hurray 1
Mmmm 1
Kitty's 4
presto 1
Aaa-ee 1
Whoa 1
Black's 1
Hell 11
oystchers'll 1
Goolick 1
He'll 14
Knife's 2
t'lah 1
Man 1
Jerusalem 1
insomma 1
Goddamn 1
They've 8
throat's 1
Indeed 2
Umm 1
Atta 2
Hey 11
Somebody'll 1
Nothing's 1
Mack's 2
All 210
damnit 1
O 10
Hello 6
Whyn't 1
Hmpf 1
Sssshoo 1
Hooray 1
They're 28
Ahah 1
diddle 1
eh 3
granite's 1
well 22
That's 97
Say 1
shucks 1
Good-by 1
Arthur's 1
She'd 13
godamit 1
uh-huh 1
sky's 2
Who's 10
hush 1
sun's 1
what's 15
please 8
Off 4
Heat's 1
God 2
Shh 1
camera's 1
over 383
amen 5
pugh 1
money's 1
leg's 1
goodnight 2
Down 2
okay 2
Bullshit 1
we'll 26
How's 9
Them's 2
dammit

radioactive 10
ghostly 2
immutable 1
self-judging 1
Theological 3
homebound 1
gowned 1
life-death 1
analeptic 2
swanlike 1
overhead 7
Systemic 2
oversize 2
Loud 1
eighth 17
Residential 4
lurid 3
innermost 1
fringed-wrapped 1
paranormal 1
straight-backed 1
S-11 1
buccolic 1
physical-chemical 1
unchristian 1
fine-chiseled 1
floppy 1
oblique 1
wily 2
by-passing 1
two-game 1
six-foot 2
memorial 2
wiser 7
selling 1
deductive 3
cross-eyed 1
baseless 1
sq. 4
irresolvable 1
unwrinkled 1
prehistoric 2
uncousinly 1
black-clad 1
drunker 2
larger 122
noxious 2
rotund 1
libelous 1
Carolingian 1
papillary 1
sway-backed 1
Einsteinian 1
Past 1
immeasurable 2
16-mesh 1
gilt 2
unsatisfactory 8
fore 1
graduate 4
double-breasted 2
Heavy 3
creedal 1
once-in-a-lifetime 1
addle-brained 1
Western-style 1
presumptuous 4
unmalicious 1
territorial 12
Ithacan 1
inertial 3
vindictive 2
tolerant 9
sun-bleached 1
bittersweet 1
Colonial 7
folksy 3
hypophyseal 1
world-ignoring 1
masterly 1
sterling 2
bogus 3
American-

...

idea-exchange 1
child-face 1
alkalis 2
endosperm 1
deceit 2
Gilkson 1
Westhampton 1
pedigree 3
Burton's 2
metalsmiths 1
parimutuels 1
timeliness 2
Robertsons 1
Conflict 1
member 133
Roleplaying 3
Ebbetts 1
pilot's 2
boldness 3
patsy 1
Illinois' 1
deed 7
newsboy 2
tappets 12
double-crossing 1
operetta 6
bass 15
Hypotheses 2
asbestos-cement 1
inns 1
blue 15
knife 73
Disquisition 1
quarry 7
jade 1
Vienna 22
Milk 5
biopsy 2
deer 8
sub-chiefdom 1
blocks 37
saturation 5
chlorine-carbon 1
Motel 4
tutor 4
early-season 1
Ormoc 2
snorkle 2
dislikes 1
sun-suit 1
post-mortem 1
mills 11
Seymour 1
fund-raisers 1
seminarians 1
Mushr 1
connection 68
Varnessa 1
favorites 10
complainant 1
Morgan 72
Science 26
Judson 3
complaint 14
blanket 29
enrollment 6
floorboards 4
Vagabonds 1
creases 1
diamonds 7
wrath 8
sweets 2
wealth 20
Rum 1
circumlocution 1
de-iodinase 1
Strindberg 1
flash-bulbs 1
blinkers 1
O'Hare 1
Bermuda 9
Palestine 7
Laughlin 2
Bosis' 1
side-arm 1
wrought-iron 1
entropy 4
Boys 11
proficien

pebbles 3
bark 13
reflector 5
integration 47
pecs 3
beer 32
tularemia 2
Eshleman 1
Hume's 2
look-see 1
vibrato 1
Lante 1
surge 8
Angelina 4
lucy 1
mister 3
reflex 3
chops 3
Silk 1
Perier 6
restlessness 2
Pilgrim's 1
Raoul 2
y 1
Arrack 1
Eckart 1
nation's 32
Jolla 2
enzymes 11
schedule 34
beggar 2
Arrington 1
chowders 1
outposts 1
butterfat 1
Orioles' 1
equivalent-choice 1
hens' 1
confidences 1
itinerary 3
transposition 1
Vernava 3
schnooks 1
merry-go-round 1
Mommor 1
Flats 1
plainclothes 1
elan 1
canals 1
cubes 4
demolition 1
Boils 1
ethers 1
Asch 1
Catalog 2
classification 21
Field's 2
fish 31
launch 3
catalyst 3
Hayes 5
Lorde 1
eulogizers 1
freights 1
1% 2
ineffectiveness 1
textiles 15
plagiarism 1
once-over 1
P 47
high-velocity 1
Collinsville 1
Peru 4
glover 1
Gatlinburg 2
Pensacola 4
hour's 1
rolls 8
Requests 3
Osaka 6
reorganizations 2
Huxley 6
wreath 8
Cedvet 1
Morse's 5
Merc 2
Travellers 1
20-megaton 1
ocher 1
**yb 3
Bayanihan 1
gastronomy 1
transcription 2
Bellini 3
Jouvet 4
va

cobalt-60 1
granite 2
road-circuit 1
boat-yard 1
Culbertson 1
monograph 1
Yancy-6 1
rocket-bombs 1
lagers 1
april 1
squires 1
Cen-Tennial 2
crudity 1
broods 3
employees 65
foot 68
County's 1
couple's 1
secularism 1
Federalist 1
fritters 1
Rawlins 3
voters 18
father-brother 1
Marlin 2
acclaim 4
radiocarbon 1
jump 8
doorbell 2
raising 8
Guiana 2
assists 1
porches 2
inventory 20
metrazol 1
Torpetius 1
bigotry 2
windowpanes 2
Johnson's 10
planes 24
carpeting 2
Beman 1
coatings 12
pre-selling 1
economists 5
mothers' 3
ulcerations 1
Sparrow-size 1
feat 6
paunch 2
Canal 3
curiosity 23
t's 1
undoing 1
cartons 1
Rebecca 1
chairing 1
Valentine 2
Yankees 28
helmsman 1
glands 6
manual 1
Martin's 1
Tiao 1
concept 85
handhold 1
jaws 10
Catskill 5
Mahler's 4
resolution 62
passport 6
fears 43
runabout 1
10-degrees-C 1
Square's 1
snake 42
1-inch 1
neutrality 3
colossus 1
Lemon 4
Napoleon 7
canyonside 1
Verdi's 2
sincerity 13
Physique 1
Borough 2
Vienot 1
Hewlett-Woodmere 1
Burke's 1
larceny 2
Gizenga 2

ability 74
Cole 1
cubist 1
Nathanael 1
polymerizations 1
Tannhaeuser 2
grease 9
RA 1
loader 1
Pasadena's 1
Auditorium 6
embarrassment 8
cabs 1
Karamazov 4
depressions 3
noun 1
Gortonists 2
formability 1
Wildlife 1
Allay 1
auditors 3
Moriarty 5
acumen 1
poetry-and-jazz 4
Ada's 4
Stephane 1
misfortunes 1
Industry's 1
sienna 2
larder 1
movers 4
creak 1
Hubie's 1
Robby's 1
specificity 10
turnips 1
disunion 1
baseman 3
TNT 4
surfaceness 1
calisthenics 4
brunches 1
Urielites 1
conducts 1
ex-Yankee 1
courtliness 1
Spillane's 1
Zemlinsky 2
F. 59
Herford 5
watching 2
poppy 2
yoga 1
grant 22
sorrow 9
Rockport 2
buyers 5
nymphs 1
Fellowship 10
confidentiality 1
unawareness 3
rediscovery 2
Height 1
hoste 1
Luthuli 1
foal 2
foaming 4
attendants 7
Lescaut 1
sect 2
Toll 2
quibs 1
widower 1
Ziminska-Sygietynska 1
Spanish 5
sheriffs 3
alizarin 1
Curie 2
Tuberculosis 1
Trapp 1
phosphate 7
suey 1
sultans 3
Grafton 5
Carruthers 5
turtle-neck 1
Albania 3
Danaher 2
spoonful 1
shop 49
Attic 1
rulers' 1
madme

Houston 25
sprays 1
Bath 1
mulch 6
salesman 12
Melamine 2
lies 3
kitchen 90
kindergarten 3
almonds 2
nomenclature 7
stacks 1
Tire 2
arrangers 1
exposition 4
Orcutt 2
Mike 91
prosperity 12
concentration 47
Suzuki 1
Lauri 1
Diseases 3
handshake 1
continence 1
City 134
cards 33
Proxmire 1
dismemberment 2
pause 16
wines 24
Brumidi's 6
starter 2
Hanover-Sally 1
gasps 5
Djakarta 1
currency 7
gasser 1
dower 1
lambs 7
Heraclitus 1
comparisons 6
fall's 1
rival's 1
sentinel 2
headlines 7
configuration 7
Turnout 1
gangs 6
hypervelocity 1
Krumpp 1
Epsom 1
porch 42
constables 1
antibodies 9
Swift 20
imminence 1
carbon 28
parenthood 3
furlough 2
Arco 1
Lublin 15
Boredom 1
Scrapiron 1
airplanes 10
Ward's 1
D-night 1
tight-turn 1
Sunny 1
Caldwell's 2
Alsing 1
interferometer 4
Feuchtwanger 2
SETSW 1
enterprise 31
gain 50
resorcinol 3
judging 2
territories 8
sandals 5
combinations 19
Roomberg 1
Jockey 2
AMA 1
Bench 7
Connally 8
abeyance 3
Quick-Wate 1
mite-box 1
young 2
wrongdoing 2
suitor 1
simpleton 1

In [221]:
for tag in data.tagset:
    

ADP
DET
NUM
CONJ
PRT
PRON
.
X
ADJ
VERB
NOUN
ADV


In [219]:
tag_unigrams

{'.': 147565,
 'ADJ': 83721,
 'ADP': 144766,
 'ADV': 56239,
 'CONJ': 38151,
 'DET': 137019,
 'NOUN': 275558,
 'NUM': 14874,
 'PRON': 49334,
 'PRT': 29829,
 'VERB': 182750,
 'X': 1386}

In [216]:
for 
basic_model.states()

TypeError: 'NoneType' object is not callable

In [205]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")

# TODO: create states with emission probability distributions P(word | tag) and add to the model
# (Hint: you may need to loop & create/add new states)
basic_model.add_states()

# TODO: add edges between states for the observed transition frequencies P(tag_i | tag_i-1)
# (Hint: you may need to loop & add transitions
basic_model.add_transition()


# NOTE: YOU SHOULD NOT NEED TO MODIFY ANYTHING BELOW THIS LINE
# finalize the model
basic_model.bake()

assert all(tag in set(s.name for s in basic_model.states) for tag in data.training_set.tagset), \
       "Every state in your network should use the name of the associated tag, which must be one of the training set tags."
assert basic_model.edge_count() == 168, \
       ("Your network should have an edge from the start node to each state, one edge between every " +
        "pair of tags (states), and an edge from each state to the end node.")
HTML('<div class="alert alert-block alert-success">Your HMM network topology looks good!</div>')

TypeError: add_transition() takes at least 3 positional arguments (0 given)

In [ ]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

assert hmm_training_acc > 0.97, "Uh oh. Your HMM accuracy on the training set doesn't look right."
assert hmm_testing_acc > 0.955, "Uh oh. Your HMM accuracy on the testing set doesn't look right."
HTML('<div class="alert alert-block alert-success">Your HMM tagger accuracy looks correct! Congratulations, you\'ve finished the project.</div>')

### Example Decoding Sequences with the HMM Tagger

In [ ]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")


## Finishing the project
---

<div class="alert alert-block alert-info">
**Note:** **SAVE YOUR NOTEBOOK**, then run the next cell to generate an HTML copy. You will zip & submit both this file and the HTML copy for review.
</div>

In [ ]:
!!jupyter nbconvert *.ipynb

## Step 4: [Optional] Improving model performance
---
There are additional enhancements that can be incorporated into your tagger that improve performance on larger tagsets where the data sparsity problem is more significant. The data sparsity problem arises because the same amount of data split over more tags means there will be fewer samples in each tag, and there will be more missing data  tags that have zero occurrences in the data. The techniques in this section are optional.

- [Laplace Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) (pseudocounts)
    Laplace smoothing is a technique where you add a small, non-zero value to all observed counts to offset for unobserved values.

- Backoff Smoothing
    Another smoothing technique is to interpolate between n-grams for missing data. This method is more effective than Laplace smoothing at combatting the data sparsity problem. Refer to chapters 4, 9, and 10 of the [Speech & Language Processing](https://web.stanford.edu/~jurafsky/slp3/) book for more information.

- Extending to Trigrams
    HMM taggers have achieved better than 96% accuracy on this dataset with the full Penn treebank tagset using an architecture described in [this](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf) paper. Altering your HMM to achieve the same performance would require implementing deleted interpolation (described in the paper), incorporating trigram probabilities in your frequency tables, and re-implementing the Viterbi algorithm to consider three consecutive states instead of two.

### Obtain the Brown Corpus with a Larger Tagset
Run the code below to download a copy of the brown corpus with the full NLTK tagset. You will need to research the available tagset information in the NLTK docs and determine the best way to extract the subset of NLTK tags you want to explore. If you write the following the format specified in Step 1, then you can reload the data using all of the code above for comparison.

Refer to [Chapter 5](http://www.nltk.org/book/ch05.html) of the NLTK book for more information on the available tagsets.

In [ ]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import brown

nltk.download('brown')
training_corpus = nltk.corpus.brown
training_corpus.tagged_sents()[0]